<a href="https://colab.research.google.com/github/miatsivitse/Project-4/blob/miatsivitse/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing

In [1]:
#import dependencies
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import tensorflow as tf

#set up connection to databse
from sqlalchemy import create_engine


protocol = 'postgresql'
username = 'USERNAME'
password = 'USER PW'
host = 'wine.corp16oy9wyq.us-east-1.rds.amazonaws.com'
port = 5432
database = 'wine'

rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database}'
        
engine = create_engine(rds_connection_string)
conn = engine.connect()

In [3]:
#Retrieve Data from RDS SQL Database
data = pd.read_sql("SELECT * FROM wine_data", conn)
data.head()

,wine_id,country,description,designation,points,price,province,region_1,region_2,taster_name,...,max_temp,precip,humidity,heat_index,latitude_y,longitude_y,elevation,review_neg,review_neu,review_pos
0,1,US,"Overripe and Porty, with raisin, prune and cho...",Reserve,81,37.0,California,Paso Robles,Central Coast,None,...,100.4,7.94,65.69,101.7,33.343447,-117.203325,190.0,0.000,0.829,0.171
1,2,US,"Strong aromas of blueberry paste, cracked pepp...",Paso Bordo Reserve,92,69.0,California,Paso Robles,Central Coast,Matt Kettmann,...,100.4,7.94,65.69,101.7,33.343447,-117.203325,190.0,0.000,0.805,0.195
2,3,US,A vegetal note drags down the enjoyment. On th...,Reserve,84,39.0,California,Paso Robles,Central Coast,None,...,100.4,7.94,65.69,101.7,33.343447,-117.203325,190.0,0.129,0.710,0.162
3,4,US,Larry Stanton patiently waits to release the w...,La Terraza Bloc Reserve,93,59.0,California,Paso Robles,Central Coast,Matt Kettmann,...,100.4,7.94,65.69,101.7,33.343447,-117.203325,190.0,0.039,0.865,0.096
4,5,US,"High alcohol gives the wine heat, especially i...",La Tertazza Bloc Reserve,85,39.0,California,Paso Robles,Central Coast,None,...,100.4,7.94,65.69,101.7,33.343447,-117.203325,190.0,0.000,0.895,0.105


In [7]:
print(data.nunique())


wine_id                  48847
country                      1
description              45206
designation              12744
points                      21
price                      158
province                     4
region_1                   201
region_2                    17
taster_name                 13
taster_twitter_handle       12
title                    45026
variety                    252
winery_name               4803
latitude_x                3032
longitude_x               3000
min_temp                   360
max_temp                   261
precip                    1564
humidity                  1306
heat_index                 242
latitude_y                2973
longitude_y               3185
elevation                  936
review_neg                 254
review_neu                 476
review_pos                 465
dtype: int64


In [8]:
data.drop(["wine_id", "country", "description", "designation","taster_name", "latitude_y", "longitude_y", "taster_twitter_handle", "title" ], axis=1, inplace=True)
data.head()

,points,price,province,region_1,region_2,variety,winery_name,latitude_x,longitude_x,min_temp,max_temp,precip,humidity,heat_index,elevation,review_neg,review_neu,review_pos
0,81,37.0,California,Paso Robles,Central Coast,Merlot,Cerro Prieto winery,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.0,0.000,0.829,0.171
1,92,69.0,California,Paso Robles,Central Coast,Cabernet Sauvignon,Cerro Prieto winery,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.0,0.000,0.805,0.195
2,84,39.0,California,Paso Robles,Central Coast,Pinot Noir,Cerro Prieto winery,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.0,0.129,0.710,0.162
3,93,59.0,California,Paso Robles,Central Coast,Syrah,Cerro Prieto winery,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.0,0.039,0.865,0.096
4,85,39.0,California,Paso Robles,Central Coast,Syrah,Cerro Prieto winery,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.0,0.000,0.895,0.105


In [9]:
#create bins for price ranges
def price_to_range(price): 
  if (price < 20): 
    return 0 
  elif (price >20) and (price <= 45):
    return 1
  if (price >45) and (price <= 80):
    return 2 
  if (price >80) and (price <= 100):
    return 3
  if (price >140) and (price <= 300): 
    return 4
  else: 
    return 5
  
data["price"] = data["price"].apply(price_to_range)
data = data.rename(columns={"price": "price_range"})
data.head()

,points,price_range,province,region_1,region_2,variety,winery_name,latitude_x,longitude_x,min_temp,max_temp,precip,humidity,heat_index,elevation,review_neg,review_neu,review_pos
0,81,1,California,Paso Robles,Central Coast,Merlot,Cerro Prieto winery,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.0,0.000,0.829,0.171
1,92,2,California,Paso Robles,Central Coast,Cabernet Sauvignon,Cerro Prieto winery,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.0,0.000,0.805,0.195
2,84,1,California,Paso Robles,Central Coast,Pinot Noir,Cerro Prieto winery,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.0,0.129,0.710,0.162
3,93,2,California,Paso Robles,Central Coast,Syrah,Cerro Prieto winery,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.0,0.039,0.865,0.096
4,85,1,California,Paso Robles,Central Coast,Syrah,Cerro Prieto winery,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.0,0.000,0.895,0.105


In [10]:
def points_to_range(points): 
  if (points >= 90):
    return 0
  else: 
    return 1
  
data["points"] = data["points"].apply(points_to_range)
data = data.rename(columns={"points": "point_range"})
data.head()

,point_range,price_range,province,region_1,region_2,variety,winery_name,latitude_x,longitude_x,min_temp,max_temp,precip,humidity,heat_index,elevation,review_neg,review_neu,review_pos
0,1,1,California,Paso Robles,Central Coast,Merlot,Cerro Prieto winery,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.0,0.000,0.829,0.171
1,0,2,California,Paso Robles,Central Coast,Cabernet Sauvignon,Cerro Prieto winery,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.0,0.000,0.805,0.195
2,1,1,California,Paso Robles,Central Coast,Pinot Noir,Cerro Prieto winery,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.0,0.129,0.710,0.162
3,0,2,California,Paso Robles,Central Coast,Syrah,Cerro Prieto winery,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.0,0.039,0.865,0.096
4,1,1,California,Paso Robles,Central Coast,Syrah,Cerro Prieto winery,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.0,0.000,0.895,0.105


In [11]:
variety_values = data["variety"].value_counts()
variety_value_counts = variety_values[variety_values > 100]
variety_value_counts

Pinot Noir                  8868
Cabernet Sauvignon          6811
Chardonnay                  6241
Syrah                       2978
Red Blend                   2548
Zinfandel                   2515
Merlot                      2137
Sauvignon Blanc             1891
Bordeaux-style Red Blend    1664
Riesling                    1550
Cabernet Franc               830
Pinot Gris                   764
RosÃ©                        701
RhÃ´ne-style Red Blend       649
Petite Sirah                 643
Viognier                     640
Sparkling Blend              608
White Blend                  523
Malbec                       491
Grenache                     489
Sangiovese                   365
Pinot Grigio                 321
Tempranillo                  299
GewÃ¼rztraminer              279
Meritage                     215
Barbera                      203
RhÃ´ne-style White Blend     198
MourvÃ¨dre                   181
Pinot Blanc                  178
Petit Verdot                 171
G-S-M     

In [12]:
# Choose a cutoff value and create a list of varieties to be replaced
cutoff = 1500
variety_types_to_replace = list(variety_values[variety_values <= cutoff].index)

# Replace in dataframe
for variety in variety_types_to_replace:
   data['variety'] = data['variety'].replace(variety,"Other")

# Check to make sure binning was successful
data['variety'].value_counts()

Other                       11644
Pinot Noir                   8868
Cabernet Sauvignon           6811
Chardonnay                   6241
Syrah                        2978
Red Blend                    2548
Zinfandel                    2515
Merlot                       2137
Sauvignon Blanc              1891
Bordeaux-style Red Blend     1664
Riesling                     1550
Name: variety, dtype: int64

In [13]:
winery_values = data["winery_name"].value_counts()
winery_value_counts = winery_values[winery_values > 100]
winery_value_counts

Testarossa winery               217
Williams Selyem winery          198
Chateau Ste. Michelle winery    193
Columbia Crest winery           158
Siduri winery                   126
Gary Farrell winery             125
Kendall-Jackson winery          118
Lynmar winery                   118
Robert Mondavi winery           112
Fess Parker winery              108
Iron Horse winery               106
Foxen winery                    103
Maryhill winery                 102
Chehalem winery                 102
Kunde winery                    101
Name: winery_name, dtype: int64

In [14]:
data.drop(['winery_name'], axis=1, inplace=True)
data.columns

Index(['point_range', 'price_range', 'province', 'region_1', 'region_2',
       'variety', 'latitude_x', 'longitude_x', 'min_temp', 'max_temp',
       'precip', 'humidity', 'heat_index', 'elevation', 'review_neg',
       'review_neu', 'review_pos'],
      dtype='object')

In [15]:
# Convert categorical data to numeric with `pd.get_dummies`
X_dummies = pd.get_dummies(data)
X_dummies

,point_range,price_range,latitude_x,longitude_x,min_temp,max_temp,precip,humidity,heat_index,elevation,...,variety_Cabernet Sauvignon,variety_Chardonnay,variety_Merlot,variety_Other,variety_Pinot Noir,variety_Red Blend,variety_Riesling,variety_Sauvignon Blanc,variety_Syrah,variety_Zinfandel
0,1,1,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.000,...,0,0,1,0,0,0,0,0,0,0
1,0,2,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.000,...,1,0,0,0,0,0,0,0,0,0
2,1,1,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.000,...,0,0,0,0,1,0,0,0,0,0
3,0,2,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.000,...,0,0,0,0,0,0,0,0,1,0
4,1,1,33.343447,-117.203325,34.4,100.4,7.94,65.69,101.7,190.000,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48842,1,1,47.766300,-122.148337,15.4,102.5,27.15,74.02,105.2,0.115,...,0,0,0,1,0,0,0,0,0,0
48843,1,5,46.068561,-118.342366,-0.9,115.0,16.61,57.52,106.4,0.073,...,0,0,0,0,0,0,0,1,0,0
48844,1,1,46.557883,-120.660862,2.0,112.9,6.90,57.53,107.1,0.197,...,0,0,0,1,0,0,0,0,0,0
48845,1,5,46.003999,-118.371124,-0.8,114.5,17.07,57.74,106.0,0.103,...,0,0,0,0,0,1,0,0,0,0


In [16]:
# Split our preprocessed data into our features and target arrays
X = X_dummies.drop('point_range',axis=1).values
y = data['point_range'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [17]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
len(X_train[0])

245

In [28]:
# create minmaxscaler instance
min_max_scaler = MinMaxScaler()

x_minmax = min_max_scaler.fit(X_train)

x_mm_train = x_minmax.transform(X_train)
x_mm_test = x_minmax.transform(X_test)

In [30]:
# # Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(x_mm_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

382/382 - 1s - loss: 0.6923 - accuracy: 0.5287 - 998ms/epoch - 3ms/step
Loss: 0.6922512650489807, Accuracy: 0.5286603569984436


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.99)
pca_data = pca.fit_transform(X_dummies)
pca_data_df=pd.DataFrame(pca_data)
pca_data_df

,0,1
0,65.540249,-10.160692
1,65.540147,-10.173510
2,65.540231,-10.163589
3,65.540234,-10.171511
4,65.540299,-10.161083
...,...,...
48842,-124.791398,5.886388
48843,-124.799162,13.243067
48844,-124.662154,7.192520
48845,-124.766351,13.429686


In [ ]:
pca_data_df.shape

(36635, 207)

# Machine Learning

In [19]:
input_features = len(X_train[0])
hidden_layer1 = 400
hidden_layer2= 400
hidden_layer3= 100
# nodes3 =
nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer1, activation="relu", input_dim=input_features))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer2, activation="relu"))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer3, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 400)               98400     
                                                                 
 dense_1 (Dense)             (None, 400)               160400    
                                                                 
 dense_2 (Dense)             (None, 100)               40100     
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 299,001
Trainable params: 299,001
Non-trainable params: 0
_________________________________________________________________


In [20]:
# # Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# # Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
1145/1145 [==============================] - 8s 6ms/step - loss: 0.5906 - accuracy: 0.6831
Epoch 2/100
1145/1145 [==============================] - 6s 5ms/step - loss: 0.5518 - accuracy: 0.7131
Epoch 3/100
1145/1145 [==============================] - 7s 6ms/step - loss: 0.5402 - accuracy: 0.7205
Epoch 4/100
1145/1145 [==============================] - 6s 5ms/step - loss: 0.5337 - accuracy: 0.7251
Epoch 5/100
1145/1145 [==============================] - 7s 6ms/step - loss: 0.5284 - accuracy: 0.7290
Epoch 6/100
1145/1145 [==============================] - 6s 5ms/step - loss: 0.5251 - accuracy: 0.7284
Epoch 7/100
1145/1145 [==============================] - 7s 6ms/step - loss: 0.5209 - accuracy: 0.7332
Epoch 8/100
1145/1145 [==============================] - 6s 5ms/step - loss: 0.5170 - accuracy: 0.7341
Epoch 9/100
1145/1145 [==============================] - 7s 6ms/step - loss: 0.5137 - accuracy: 0.7373
Epoch 10/100
1145/1145 [==============================] - 6s 5ms/step - l

In [ ]:
fit_mm_model = nn.fit(x_mm_train ,y_train,epochs=100)

Epoch 1/100
1145/1145 [==============================] - 6s 5ms/step - loss: 0.5575 - accuracy: 0.7064
Epoch 2/100
1145/1145 [==============================] - 6s 6ms/step - loss: 0.5261 - accuracy: 0.7286
Epoch 3/100
1145/1145 [==============================] - 6s 5ms/step - loss: 0.5148 - accuracy: 0.7338
Epoch 4/100
1145/1145 [==============================] - 7s 6ms/step - loss: 0.5046 - accuracy: 0.7407
Epoch 5/100
1145/1145 [==============================] - 6s 5ms/step - loss: 0.4960 - accuracy: 0.7473
Epoch 6/100
1145/1145 [==============================] - 7s 6ms/step - loss: 0.4887 - accuracy: 0.7512
Epoch 7/100
1145/1145 [==============================] - 6s 5ms/step - loss: 0.4811 - accuracy: 0.7544
Epoch 8/100
1145/1145 [==============================] - 7s 6ms/step - loss: 0.4743 - accuracy: 0.7579
Epoch 9/100
1145/1145 [==============================] - 6s 5ms/step - loss: 0.4676 - accuracy: 0.7623
Epoch 10/100
1145/1145 [==============================] - 6s 6ms/step - l

In [22]:
# # Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

382/382 - 1s - loss: 0.6973 - accuracy: 0.5719 - 1s/epoch - 4ms/step
Loss: 0.6973086595535278, Accuracy: 0.571896493434906


In [21]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=400,
        step=20), activation=activation, input_dim=len(X_train[0])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=400,
            step=20),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [22]:
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=15,
    hyperband_iterations=2)

In [23]:
tuner.search(X_train_scaled,y_train,epochs=60,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 01m 24s]
val_accuracy: 0.5769734978675842

Best val_accuracy So Far: 0.7283819317817688
Total elapsed time: 00h 35m 10s


In [25]:
best_model = tuner.get_best_models(1)[0]

In [26]:
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

382/382 - 1s - loss: 0.5520 - accuracy: 0.7284 - 836ms/epoch - 2ms/step
Loss: 0.5519906878471375, Accuracy: 0.7283819317817688


In [27]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 221,
 'num_layers': 1,
 'units_0': 81,
 'units_1': 81,
 'units_2': 121,
 'tuner/epochs': 15,
 'tuner/initial_epoch': 5,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0023'}

# Saving Model

In [ ]:
# Export our model to HDF5 file
nn.save('../WineMLModel.h5')